In [1]:
import tensorflow as tf
import keras
import keras_nlp

EMBEDDING_DIM = 64
NUM_HEADS = 2
INTERMIDIATE_DIM = 256

dataset = [
    (
        "The quick brown fox jumps over the lazy dog.",
        "<start> A fox jumps over a dog.",
        "A fox jumps over a dog. <end>",
    ),
    (
        "An apple a day keeps the doctor away.",
        "<start> Eating an apple daily is healthy.",
        "Eating an apple daily is healthy. <end>",
    ),
]

# Preprocessing
input_texts, target_texts, decoder_target_text = zip(*dataset)

tokenizer = keras.preprocessing.text.Tokenizer(
    split=' ',
    filters='!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n'
)
tokenizer.fit_on_texts(input_texts + target_texts + decoder_target_text)
vocab_size = len(tokenizer.word_index) + 1

input_sequences = tokenizer.texts_to_sequences(input_texts)
target_sequences = tokenizer.texts_to_sequences(target_texts)
decoder_target_sequences = tokenizer.texts_to_sequences(decoder_target_text)

max_input_length = max(len(seq) for seq in input_sequences)
max_target_length = max(len(seq) for seq in target_sequences)

input_sequences = keras.preprocessing.sequence.pad_sequences(
    input_sequences,
    maxlen=max_input_length,
    padding='post'
)
target_sequences = keras.preprocessing.sequence.pad_sequences(
    target_sequences,
    maxlen=max_target_length,
    padding='post'
)
decoder_target_sequences = keras.preprocessing.sequence.pad_sequences(
    decoder_target_sequences,
    maxlen=max_target_length,
    padding='post'
)
decoder_target_sequences = tf.expand_dims(decoder_target_sequences, axis=-1)

# Build model
encoder_inputs = keras.Input(
    shape=(max_input_length,),
    name="encoder_inputs"
)
encoder_embedding = keras_nlp.layers.TokenAndPositionEmbedding(
    vocabulary_size=vocab_size,
    sequence_length=max_input_length,
    embedding_dim=EMBEDDING_DIM,
)(encoder_inputs)
encoder_outputs = keras_nlp.layers.TransformerEncoder(
    num_heads=NUM_HEADS,
    intermediate_dim=INTERMIDIATE_DIM,
)(encoder_embedding)

decoder_inputs = keras.Input(
    shape=(max_target_length,),
    name="decoder_inputs"
)
decoder_embedding = keras_nlp.layers.TokenAndPositionEmbedding(
    vocabulary_size=vocab_size,
    sequence_length=max_target_length,
    embedding_dim=EMBEDDING_DIM,
)(decoder_inputs)
decoder_outputs = keras_nlp.layers.TransformerDecoder(
    num_heads=NUM_HEADS,
    intermediate_dim=INTERMIDIATE_DIM,
)(decoder_embedding, encoder_outputs)

outputs = keras.layers.Dense(
    vocab_size,
    activation="softmax"
)(decoder_outputs)

model = keras.Model(
    [encoder_inputs, decoder_inputs],
    outputs
)
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

# Training
model.fit(
    [input_sequences, target_sequences],
    decoder_target_sequences,
    epochs=100,
)


Using TensorFlow backend


2024-06-04 00:28:27.688743: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2 Pro
2024-06-04 00:28:27.688762: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 32.00 GB
2024-06-04 00:28:27.688766: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 10.67 GB
2024-06-04 00:28:27.688792: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-06-04 00:28:27.688809: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/100


2024-06-04 00:28:30.614040: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 2s 2s/step - loss: 4.0426 - accuracy: 0.1429
Epoch 2/100
1/1 [==============================] - 0s 39ms/step - loss: 3.0763 - accuracy: 0.2857
Epoch 3/100
1/1 [==============================] - 0s 32ms/step - loss: 2.3059 - accuracy: 0.3571
Epoch 4/100
1/1 [==============================] - 0s 30ms/step - loss: 1.7173 - accuracy: 0.5714
Epoch 5/100
1/1 [==============================] - 0s 29ms/step - loss: 1.2740 - accuracy: 0.7143
Epoch 6/100
1/1 [==============================] - 0s 29ms/step - loss: 0.9359 - accuracy: 0.9286
Epoch 7/100
1/1 [==============================] - 0s 30ms/step - loss: 0.6747 - accuracy: 0.8571
Epoch 8/100
1/1 [==============================] - 0s 30ms/step - loss: 0.4820 - accuracy: 1.0000
Epoch 9/100
1/1 [==============================] - 0s 30ms/step - loss: 0.3450 - accuracy: 1.0000
Epoch 10/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2518 - accuracy: 1.0000
Epoch 11/100
1/1 [===============

In [2]:
# テストデータによる予測
def summarize(text):
    input_seq = tokenizer.texts_to_sequences([text])
    input_seq = tf.keras.preprocessing.sequence.pad_sequences(
        input_seq,
        maxlen=max_input_length,
        padding='post'
    )
    idx = tokenizer.word_index['<start>']
    decoder_input_seq = tf.constant([[idx]], dtype=tf.int64)
    
    summary = []
    for _ in range(max_target_length):
        preds = model.predict([input_seq, decoder_input_seq], verbose=0)
        next_token = tf.argmax(preds[0, -1, :])
        next_word = tokenizer.index_word.get(next_token.numpy(), '<unk>')
        if next_word == '<end>':
            break
        summary.append(next_word)
        decoder_input_seq = tf.concat(
            [decoder_input_seq, tf.expand_dims([next_token], axis=-1)],
            axis=-1
        )
    return ' '.join(summary)

# サンプルテキストの要約
sample_text = "The quick brown fox jumps over the lazy dog."
print("Original text:", sample_text)
print("Summary:", summarize(sample_text))

sample_text = "An apple a day keeps the doctor away."
print("Original text:", sample_text)
print("Summary:", summarize(sample_text))

sample_text = "A quick apple a day keeps the dog away."
print("Original text:", sample_text)
print("Summary:", summarize(sample_text))


Original text: The quick brown fox jumps over the lazy dog.
Summary: a fox jumps over a dog
Original text: An apple a day keeps the doctor away.
Summary: eating an apple daily is healthy
Original text: A quick apple a day keeps the dog away.
Summary: eating an apple daily is healthy


In [3]:
tokenizer.word_index

{'a': 1,
 'the': 2,
 'fox': 3,
 'jumps': 4,
 'over': 5,
 'dog': 6,
 'an': 7,
 'apple': 8,
 '<start>': 9,
 'eating': 10,
 'daily': 11,
 'is': 12,
 'healthy': 13,
 '<end>': 14,
 'quick': 15,
 'brown': 16,
 'lazy': 17,
 'day': 18,
 'keeps': 19,
 'doctor': 20,
 'away': 21}